# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install openai

In [ ]:
# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

In [ ]:
import json

# JSON出力ヘルパーの準備
def show_json(obj):
    display(json.loads(obj.model_dump_json()))

# Embedding

In [ ]:
# テキストの準備
text = "これはテストです。"

In [ ]:
# テキストをEmbeddingに変換
response =client.embeddings.create(
    input=text,
    model="text-embedding-ada-002"
)
show_json(response)

In [ ]:
# Embeddingの取得
print(response.data[0].embedding)

[-0.0018969560042023659, -0.011020875535905361, -0.002355346456170082, -0.01618995890021324, -0.0077178641222417355, 0.004980525467544794, -0.021144475787878036, -0.013485130853950977, 0.00506505137309432, -0.024655552580952644, 0.00390444602817297, 0.01513663586229086, -0.0017701671458780766, -0.030741415917873383, -0.0024561272002756596, -0.009889529086649418, 0.022730963304638863, -0.012418803758919239, 0.013355090282857418, -0.03789360821247101, 0.0186087004840374, -0.0008346931426785886, -0.012444811873137951, 0.0005120318382978439, -0.03750348836183548, 0.0011817369377240539, 0.018010515719652176, -0.034174468368291855, 0.004024732857942581, -0.01793249323964119, 0.024408476427197456, -0.007795887999236584, -0.015396716073155403, -0.021664636209607124, 0.005711999721825123, -0.02375827729701996, 0.00788041390478611, -0.002226932207122445, -0.0002578446001280099, 0.006755569018423557, 0.024811599403619766, -0.005685991607606411, 0.012327776290476322, -0.0043400791473686695, 0.0049

In [ ]:
# Embeddingの長さの確認
print(len(response.data[0].embedding))

1536


# 近傍探索

In [ ]:
# Faissパッケージのインストール
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.6 MB/s eta 0:00:00


In [ ]:
import numpy as np

# 入力テキストのEmbeddingの生成
in_text = "今日は雨振らなくてよかった"
response =client.embeddings.create(
    input=in_text,
    model="text-embedding-ada-002"
)

# numpy配列に変換
in_embeds = [record.embedding for record in response.data]
in_embeds = np.array(in_embeds).astype("float32")

In [ ]:
# 対象テキストのEmbeddingの生成
target_texts = [
    "好きな食べ物は何ですか?",
    "どこにお住まいですか?",
    "朝の電車は混みますね",
    "今日は良いお天気ですね",
    "最近景気悪いですね"]
response =client.embeddings.create(
    input=target_texts,
    model="text-embedding-ada-002"
)

# numpy配列に変換
target_embeds = [record.embedding for record in response.data]
target_embeds = np.array(target_embeds).astype("float32")

In [ ]:
import faiss

# Faissのインデックス生成
index = faiss.IndexFlatL2(1536)

In [ ]:
# 対象テキストをインデックスに追加
index.add(target_embeds)

In [ ]:
# 近傍探索の実行
D, I = index.search(in_embeds, 1)

# 確認
print(D)
print(I)
print(target_texts[I[0][0]])

[[0.17736347]]
[[3]]
今日は良いお天気ですね
